In [26]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

### Defining Setup Time and Hourly Yield Constants

In [27]:
T = pd.read_csv('setup_constants.csv')
R = pd.read_csv('yield_constants.csv')

In [28]:
T.index += 1
R.index += 1

### Defining Weekly Demand Constants

In [29]:
D = pd.read_csv('weekly_demands.csv')

In [30]:
D.index += 1

# Defining Problem

In [31]:
model = gp.Model('base_problem')

## Variables

### Adding Binary Setup Variables

In [32]:
S = []
for i in range(5):
    S_i = []
    for j in range(5):
        S_i.append(model.addVar(vtype=GRB.BINARY, name=f"S_{i}_{j}", lb=0))
    S.append(S_i)

### Adding Binary Overtime Setup Variables

In [33]:
Sp = []
for i in range(5):
    Sp_i = []
    for j in range(5):
        Sp_i.append(model.addVar(vtype=GRB.BINARY, name=f"Sp_{i}_{j}", lb=0))
    Sp.append(Sp_i)

### Adding Production Variables

In [34]:
P = []
for i in range(5):
    P_i = []
    for j in range(5):
        P_i.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"P_{i}_{j}", lb=0))
    P.append(P_i)

### Adding Overtime Production Variables

In [35]:
Pp = []
for i in range(5):
    Pp_i = []
    for j in range(5):
        Pp_i.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"Pp_{i}_{j}", lb=0))
    Pp.append(Pp_i)

## Constraints

### Demand Constraints

In [36]:
for i in range(5):
    constr_expr = gp.quicksum(P[j][i]*R.iloc[j,i] + Pp[j][i]*R.iloc[j,i] for j in range(5))
    model.addConstr(constr_expr >= D.iloc[0][i], name=f'Demand_Prod_{i+1}')

/var/folders/w_/z02fg3l56_jfc9tys40hqtdm0000gn/T/ipykernel_27623/2297407209.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  model.addConstr(constr_expr >= D.iloc[0][i], name=f'Demand_Prod_{i+1}')


### Normal Production Time Constraints

In [37]:
for i in range(5):
    constr_expr = gp.quicksum(S[i][j]*T.iloc[i,j] + P[i][j] for j in range(5))
    model.addConstr(constr_expr <= 120, name=f'Prod_Hrs_Mach_{i+1}')

### Overtime Production Time Constraints

In [38]:
for i in range(5):
    constr_expr = gp.quicksum(Sp[i][j]*T.iloc[i,j] + Pp[i][j] for j in range(5))
    model.addConstr(constr_expr <= 48, name=f'OT_Prod_Hrs_Mach_{i+1}')

### Production Setup Linkage Constants
- Regular Time and Overtime are both included here

In [39]:
for i in range(5):
    for j in range(5):
        model.addConstr(P[i][j]*R.iloc[i,j] <= 10000*S[i][j], name=f'SetupProdLinkage_Mach_{i+1}_Prod_{j+1}')
        model.addConstr(Pp[i][j]*R.iloc[i,j] <= 10000*(S[i][j]+Sp[i][j]), name=f'OTSetupProdLinkage_Mach_{i+1}_Prod_{j+1}')

## Objective Function

In [40]:
obj_parts = []
for i in range(5):
    obj_parts.append(gp.quicksum(Sp[i][j]*T.iloc[i,j] + Pp[i][j] for j in range(5)))

In [41]:
model.setObjective(gp.quicksum(obj_parts),GRB.MINIMIZE)

In [42]:
model.update()

In [43]:
model.getObjective()

<gurobi.LinExpr: 8.0 Sp_0_0 + 8.0 Sp_0_3 + 10.0 Sp_1_0 + 8.0 Sp_1_1 + 10.0 Sp_2_1 + 24.0 Sp_2_4 + 8.0 Sp_3_1 + 12.0 Sp_3_2 + 8.0 Sp_4_3 + 20.0 Sp_4_4 + Pp_0_0 + Pp_0_1 + Pp_0_2 + Pp_0_3 + Pp_0_4 + Pp_1_0 + Pp_1_1 + Pp_1_2 + Pp_1_3 + Pp_1_4 + Pp_2_0 + Pp_2_1 + Pp_2_2 + Pp_2_3 + Pp_2_4 + Pp_3_0 + Pp_3_1 + Pp_3_2 + Pp_3_3 + Pp_3_4 + Pp_4_0 + Pp_4_1 + Pp_4_2 + Pp_4_3 + Pp_4_4>

In [44]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 65 rows, 100 columns and 185 nonzeros
Model fingerprint: 0x6c1530b5
Variable types: 50 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 4e+03]
Found heuristic solution: objective 186.5483405
Presolve removed 39 rows and 69 columns
Presolve time: 0.00s
Presolved: 26 rows, 31 columns, 82 nonzeros
Found heuristic solution: objective 178.5483405
Variable types: 16 continuous, 15 integer (15 binary)

Root relaxation: objective 1.120894e+02, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

In [45]:
model.Status == GRB.OPTIMAL

True

In [46]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(S[i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 8 0 0 8 0 
Machine 2: 10 0 0 0 0 
Machine 3: 0 10 0 0 0 
Machine 4: 0 8 12 0 0 
Machine 5: 0 0 0 0 20 


In [47]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(Sp[i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 0 0 0 0 0 
Machine 2: 0 0 0 0 0 
Machine 3: 0 0 0 0 0 
Machine 4: 0 0 0 0 0 
Machine 5: 0 0 0 0 0 


In [48]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(P[i][j].X)), end=" ")
    print("")

Machine 1: 49.43589743589781 0.0 0.0 54.56410256410219 0.0 
Machine 2: 110.0 0.0 0.0 0.0 0.0 
Machine 3: 0.0 110.0 0.0 0.0 0.0 
Machine 4: 0.0 41.333333333333336 58.666666666666664 0.0 0.0 
Machine 5: 0.0 0.0 0.0 0.0 93.33333333333333 


In [49]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(Pp[i][j].X)), end=" ")
    print("")

Machine 1: 0.0 0.0 0.0 48.0 0.0 
Machine 2: 0.16849816849775207 0.0 0.0 0.0 0.0 
Machine 3: 0.0 23.595959595959197 0.0 0.0 0.0 
Machine 4: 0.0 0.0 48.0 0.0 0.0 
Machine 5: 0.0 0.0 0.0 0.0 0.0 


In [50]:
print('Final Objective Value: ', model.ObjVal)

Final Objective Value:  119.76445776445695
